In [111]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from name_dict import parse_names, file_grab
import re
import numpy as np
import itertools
import string
import operator
from textblob import TextBlob
from summa import summarizer
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram
from nltk.stem import WordNetLemmatizer
from collections import OrderedDict
import itertools

In [15]:
dict1 = {"H": ["I said this today."], "D": ["I am TRUMP."]}
dict2 = {"H": ["I said this yesterday."], "D": ["I am DRUMF."]}

In [16]:
relevant_debates = [dict1, dict2]
aggregate_dict = {}
for d in relevant_debates:
    for k, v in d.iteritems():
        if aggregate_dict.has_key(k):
            aggregate_dict[k] = aggregate_dict[k] + v
        else:
            aggregate_dict[k] = v

In [17]:
aggregate_dict

{'D': ['I am TRUMP.', 'I am DRUMF.'],
 'H': ['I said this today.', 'I said this yesterday.']}

In [18]:
blob = TextBlob('Hi my name is Scotty. I am DS.')

In [20]:
for sentence in blob.sentences:
    print sentence

Hi my name is Scotty.
I am DS.


In [50]:
def setup_agg_df(relevant_debates, defaults = ['H', 'D']):
    #aggregate dictionaries
    aggregate_dict = {}
    for d in relevant_debates:
        for k, v in d.iteritems():
            if aggregate_dict.has_key(k):
                aggregate_dict[k] = aggregate_dict[k] + v
            else:
                aggregate_dict[k] = v

    #setup df - column = candidate name, rows = sentences
    df_list = []
    column_list = []
    flag = False
    for candidate in defaults:
        sentence_list = []
        list_words = aggregate_dict[candidate]
        string_words = (' '.join(list_words))
        textblob = TextBlob(string_words)
        for sentence in textblob.sentences:
            sentence_list.append(str(sentence))
        if flag == False: 
            df = pd.DataFrame(sentence_list, columns = [candidate])
            flag = True
        else: 
            df[candidate] = sentence_list
    return df


In [51]:
topic_modeling(relevant_debates)

,H,D
0,I said this today.,I am TRUMP.
1,I said this yesterday.,I am DRUMF.


In [57]:
STOPWORDS

<WordListCorpusReader in u'/Users/scotthuhn/nltk_data/corpora/stopwords'>

In [62]:
def tokenize(text):
    regex = re.compile('<.+?>|[^a-zA-Z]')
    clean_txt = regex.sub(' ', text)
    tokens = clean_txt.split()
    lowercased = [t.lower() for t in tokens]
    no_stopwords = [w for w in lowercased if not w in stopwords.words('english')]
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmed = [wordnet_lemmatizer.lemmatize(w) for w in no_stopwords]
    return [w for w in lemmed if w]

In [63]:
line = "this is a line of text starting here."

In [67]:
' '.join(tokenize(line))

'line text starting'

In [92]:
import urllib
data = urllib.urlencode({"text": "I'm a very good boy "}) 
u = urllib.urlopen("http://text-processing.com/api/sentiment/", data)
the_page = u.read()
print the_page

{"probability": {"neg": 0.22531846855219551, "neutral": 0.084284385065714951, "pos": 0.77468153144780449}, "label": "pos"}


In [94]:
items = the_page.split(':')
imp_items = items[2:4]
output_list = []
for string1 in imp_items: 
    output_list.append(float(string1.split(',')[0]))

In [95]:
print output_list

[0.2253184685521955, 0.08428438506571495]


In [106]:
dict1 = {1 : 'hi', 3 : 'hello', 4 : 'mr.', 2 : 'you'}

In [107]:
top_5 = OrderedDict(sorted(dict1.items()))

In [108]:
top_5

OrderedDict([(1, 'hi'), (2, 'you'), (3, 'hello'), (4, 'mr.')])

In [113]:
x = 0
for key, value in top_5.iteritems():
    if x < 3:
        print key, value
    x += 1

1 hi
2 you
3 hello


In [115]:
int('hello')

ValueError: invalid literal for int() with base 10: 'hello'